In [29]:
import json, folium
import pandas as pd
import numpy as np
import re

In [30]:

df = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
df = df[['지역','면적','X좌표(WGS84)','Y좌표(WGS84)']]
df.columns = ['구별','면적','경도','위도']
df.head()

,구별,면적,경도,위도
0,중구,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,126.990377,37.550140
1,강동구,80683㎡,127.154779,37.540394
2,과천시,9132690m²,127.019846,37.426449
3,성동구,480994㎡,127.041798,37.543072
4,마포구,2284085㎡,126.878907,37.571805


In [31]:
df.dropna(subset=['면적','경도','구별'],inplace=True)
df.isna().sum().sum()


0

In [32]:
df.면적[6]
df.drop([6],inplace=True)

In [33]:
df.면적 = df.면적.str.replace('총','')

In [34]:
df.면적 = df.면적.apply(lambda x: re.split('[㎡m㎥]',x)[0])
df.면적.head()

0    2896887
1      80683
2    9132690
3     480994
4    2284085
Name: 면적, dtype: object

In [35]:
df.면적 = df.면적.astype(float).astype(int)

In [36]:
df_전체면적 = df.면적.sum()
df['구별면적비율'] = ((df['면적'] / df_전체면적) * 100).round(3)

In [37]:
df

,구별,면적,경도,위도,구별면적비율
0,중구,2896887,126.990377,37.550140,3.116
1,강동구,80683,127.154779,37.540394,0.087
2,과천시,9132690,127.019846,37.426449,9.822
3,성동구,480994,127.041798,37.543072,0.517
4,마포구,2284085,126.878907,37.571805,2.457
...,...,...,...,...,...
125,종로구,45615,127.017827,37.576442,0.049
126,중랑구,631021,127.098782,37.587716,0.679
127,중랑구,109635,127.095997,37.579762,0.118
128,마포구,140022,126.893246,37.571718,0.151


In [38]:
df.set_index('구별',inplace=True)
df.head()

,면적,경도,위도,구별면적비율
구별,,,,
중구,2896887,126.990377,37.550140,3.116
강동구,80683,127.154779,37.540394,0.087
과천시,9132690,127.019846,37.426449,9.822
성동구,480994,127.041798,37.543072,0.517
마포구,2284085,126.878907,37.571805,2.457


In [39]:
filename = 'data/seoul_geo_simple.json'
geo_data = json.load(open(filename,encoding='utf-8'))

In [40]:
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

In [41]:
gu_dict = get_text_location(geo_data)
gu_dict

{'강동구': [37.546532751824586, 127.14078218193268],
 '송파구': [37.50620055459428, 127.11197246712628],
 '강남구': [37.49679934694853, 127.0673229896189],
 '서초구': [37.47066332386016, 127.0390912771612],
 '관악구': [37.46393155080159, 126.94996289929526],
 '동작구': [37.495988209152785, 126.9530748356924],
 '영등포구': [37.51883998627326, 126.91028348590825],
 '금천구': [37.46241523438773, 126.9050257773362],
 '구로구': [37.49166908254099, 126.85870943441728],
 '강서구': [37.55703289257873, 126.83102419541086],
 '양천구': [37.529418412364095, 126.85252712500812],
 '마포구': [37.5599165285085, 126.9176866353821],
 '서대문구': [37.57597915089086, 126.94816079153975],
 '은평구': [37.61445865754635, 126.93220977883982],
 '노원구': [37.651705676071316, 127.07821750766884],
 '도봉구': [37.66711228271955, 127.03387331602003],
 '강북구': [37.646476408369075, 127.01356807286824],
 '성북구': [37.61007782092726, 127.02020916322154],
 '중랑구': [37.59847817045754, 127.09036092595524],
 '동대문구': [37.57621738848033, 127.04968775012617],
 '광진구': [37.548640

In [42]:
df.drop(index=['과천시'], inplace=True)

In [43]:
df.head()

,면적,경도,위도,구별면적비율
구별,,,,
중구,2896887,126.990377,37.550140,3.116
강동구,80683,127.154779,37.540394,0.087
성동구,480994,127.041798,37.543072,0.517
마포구,2284085,126.878907,37.571805,2.457
종로구,8948,126.978023,37.575045,0.010


In [44]:
map = folium.Map(location=[37.55,126.98], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=df.구별면적비율,        #단계구분도로 보여줄 데이터
    columns=[df.index,df.구별면적비율],      # 데이터 프레임에서 추출할 column
    fill_color='YlOrRd',       # Color map
    key_on='properties.name'
).add_to(map)
for gu_name in df.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
title_html = '<h4 align="center" style = "font-size:15px">서울시 자치구별 공원 면적 단계 구분도</h4>'
map.get_root().html.add_child(folium.Element(title_html))
map


In [45]:
df2 = pd.read_csv('../02.Pandas/pop.csv')

In [46]:
df2= df2[['구별','인구수']]

In [47]:

df = df['구별면적비율']
df

구별
중구     3.116
강동구    0.087
성동구    0.517
마포구    2.457
종로구    0.010
       ...  
종로구    0.049
중랑구    0.679
중랑구    0.118
마포구    0.151
강남구    0.169
Name: 구별면적비율, Length: 126, dtype: float64

In [48]:
df.info

<bound method Series.info of 구별
중구     3.116
강동구    0.087
성동구    0.517
마포구    2.457
종로구    0.010
       ...  
종로구    0.049
중랑구    0.679
중랑구    0.118
마포구    0.151
강남구    0.169
Name: 구별면적비율, Length: 126, dtype: float64>

In [49]:
df2.index = df2['구별']

In [50]:
df2 = df2['인구수']
df2.info

<bound method Series.info of 구별
종로구     153789
중구      131787
용산구     237285
성동구     292672
광진구     352627
동대문구    352006
중랑구     391885
성북구     440142
강북구     302563
도봉구     319373
노원구     514946
은평구     477173
서대문구    315659
마포구     378686
양천구     450487
강서구     579768
구로구     421163
금천구     244891
영등포구    400908
동작구     394364
관악구     499449
서초구     416167
강남구     537800
송파구     663965
강동구     466472
Name: 인구수, dtype: int64>

In [51]:
popu = pd.merge(df2, df, on='구별')

In [52]:
popu

,인구수,구별면적비율
구별,,
종로구,153789,0.010
종로구,153789,0.217
종로구,153789,0.106
종로구,153789,0.021
종로구,153789,0.006
...,...,...
강동구,466472,0.029
강동구,466472,0.107
강동구,466472,0.699


In [53]:
popu['인당공원면적'] = ((popu['구별면적비율'] / popu['인구수'])*10000000).round(2)

In [54]:
popu['인당공원면적']

구별
종로구     0.65
종로구    14.11
종로구     6.89
종로구     1.37
종로구     0.39
       ...  
강동구     0.62
강동구     2.29
강동구    14.98
강동구    14.58
강동구     0.58
Name: 인당공원면적, Length: 126, dtype: float64

In [55]:
map = folium.Map(location=[37.55,126.98], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=popu.인당공원면적,        #단계구분도로 보여줄 데이터
    columns=[popu.index,popu.인당공원면적],      # 데이터 프레임에서 추출할 column
    fill_color='YlOrRd',       # Color map
    key_on='properties.name'
).add_to(map)
for gu_name in popu.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
title_html = '<h4 align="center" style = "font-size:15px">서울시 자치구별 인당 공원 면적 단계 구분도</h4>'
map.get_root().html.add_child(folium.Element(title_html))
map
